<a href="https://colab.research.google.com/github/ozturkcemal/SupplyChainAnalytics/blob/main/05_Network_Design%20/02_UncapacitatedFacilityLocationNetworkDesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing library to read xlsx files
!pip install pandas openpyxl

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:


# Import libraries
import openpyxl
import pandas as pd
import numpy as np  # Import NumPy for array manipulation


# Get the file name of the uploaded file
file_name = list(uploaded.keys())[0]

# Load the workbook
workbook = openpyxl.load_workbook(file_name, data_only=True)

# Initialize a dictionary to store DataFrames for each named range
named_ranges_data = {}

# Loop through all defined names in the workbook
for name, defined_name in workbook.defined_names.items():
    destinations = list(defined_name.destinations)
    if not destinations:
        continue  # Skip if there are no destinations

    for sheet_name, cell_range in destinations:
        try:
            sheet = workbook[sheet_name]

            # If it's a single cell, handle it differently
            if ":" not in cell_range:  # Single cell (no colon in the range)
                cell_value = sheet[cell_range].value
                # Convert the single cell value into a DataFrame (1x1)
                df = pd.DataFrame([[cell_value]])

            else:
                # Retrieve the data from the specified range
                data = []
                for row in sheet[cell_range]:
                    data.append([cell.value for cell in row])

                # Convert to DataFrame
                df = pd.DataFrame(data)

            # Store the DataFrame with the named range as the key
            named_ranges_data[name] = df

        except Exception as e:
            print(f"Error processing range {name}: {e}")
            continue

# Create an array for each DataFrame, named after the DataFrame
for range_name, df in named_ranges_data.items():
    globals()[range_name] = np.array(df)  # Converts the DataFrame to a NumPy array

# Example of accessing one of the dynamically created arrays
for range_name in named_ranges_data:
    print(f"Array created for {range_name}:")
    print(globals()[range_name])
    print()


facilities = list(range(facilities.shape[0]))
print("List of facilities:", facilities)

# Create a list of indices from 0 to num_markets - 1
markets= list(range(markets.shape[1]))

# Print the resulting list
print("List of markets:", markets)


In [ ]:
#installing pyomo
!pip install -q pyomo

In [ ]:
#installing coin or
!apt-get install -y -qq coinor-cbc

In [ ]:
#importing pyomo environment
import pyomo.environ as pyo

In [ ]:
#creating a model object
model = pyo.ConcreteModel()

In [ ]:
#defining variables
model.x = pyo.Var(facilities,markets, within=pyo.Binary)
model.y = pyo.Var(facilities, within=pyo.Binary)
model.x.pprint()
model.y.pprint()

In [ ]:
# Remove any existing objective component named "obj"
if hasattr(model, 'obj'):
    model.del_component('obj')

#defining objective function
model.obj = pyo.Objective(
    expr=sum(fixedCost[s] * model.y[s] for s in facilities) +
         sum(transportCost[s][m] * model.x[s, m] for s in facilities for m in markets if transportCost[s][m]>0),
    sense=pyo.minimize
)
model.obj.pprint()

In [ ]:
#modeling the condition that if market m is served by facility m , there must be a facility at m
model.constraint_facility = pyo.ConstraintList()
for s in facilities:
  for m in markets:
    model.constraint_facility.add(model.x[s,m] <= model.y[s])
model.constraint_facility.pprint()

In [ ]:
#modeling the single sourcing (partitioning) constraint
model.constraint_singleSourcing = pyo.ConstraintList()
for m in markets:
  model.constraint_singleSourcing.add(sum(model.x[s,m] for s in facilities) == 1)
model.constraint_singleSourcing.pprint()




In [ ]:
#connecting cbc solver and printing the model
opt = pyo.SolverFactory('cbc')
model.pprint()

In [ ]:
#solve the model
opt_solution = opt.solve(model)

In [ ]:
# Print the values of the decision variables
#we are shifting the indices for presentation
print("\nFacilities opening:")
for s in facilities:
    if model.y[s].value > 0:
      print(f"{s+1}: {model.y[s].value}")

print("\nTransport (Network) links opened:")
for s in facilities:
  for m in markets:
    if model.x[s,m].value > 0:
      print(f"({s+1},{m+1}): {model.x[s,m].value}")

print(f"total cost of facility opening is {sum(fixedCost[s] * model.y[s].value for s in facilities)[0]}")
print(f"total cost of transport is {sum(transportCost[s][m] * model.x[s, m].value for s in facilities for m in markets if transportCost[s][m]>0)}")
print(f"Total cost of facility location and network design is : {model.obj()}")